In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr  7 21:25:46 2020

@author: gn-00
"""
%load_ext autoreload
%autoreload 2
%matplotlib tk
%autosave 120


#%matplotlib tk
import sys
import argparse
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange, tqdm

from hopfield1.hopfield import hopfield
import parmap



Autosaving every 120 seconds


/home/cat/anaconda3/envs/ai/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
######################################################################
######################################################################
######################################################################
#
np.random.seed(1)
imgs = [
'/home/cat/code/ArtificialIntelligenceEngines/hopfield_notebooks/data/1.png',
#'/home/cat/code/Hopfield-Network/data/2.png',
'/home/cat/code/ArtificialIntelligenceEngines/hopfield_notebooks/data/3.png',
#'/home/cat/code/Hopfield-Network/data/4.png',
'/home/cat/code/ArtificialIntelligenceEngines/hopfield_notebooks/data/5.png',
#'/home/cat/code/Hopfield-Network/data/6.png',
#'/home/cat/code/Hopfield-Network/data/7.png',
'/home/cat/code/ArtificialIntelligenceEngines/hopfield_notebooks/data/8.png',
'/home/cat/code/ArtificialIntelligenceEngines/hopfield_notebooks/data/9.png',
'/home/cat/code/ArtificialIntelligenceEngines/hopfield_notebooks/data/A.png',
'/home/cat/code/ArtificialIntelligenceEngines/hopfield_notebooks/data/B.png',
'/home/cat/code/ArtificialIntelligenceEngines/hopfield_notebooks/data/G.png'
]

LABELS = ['1','3','5','8','9','A','B','G']

iteration = 1

input_shape = (32,32)
model = hopfield(input_shape)
print('Model initialized with weights shape ',model.W.shape)

### Training Model ###
for train_data in imgs:
    #print('Start training ',train_data,'...')
    model.addTrain(train_data)
    #print(train_data,'training completed!')
print ("...DONE...")   

Model initialized with weights shape  (1024, 1024)
...DONE...


In [3]:
############################################
############################################
############################################
# plt.figure()
# for k in range(8):
#     ax=plt.subplot(2,4,k+1)
#     img = plt.imread(imgs[k])
#     img = np.mean(img,axis=2)
#     plt.imshow(img, cmap='binary')
# plt.show()


# plt.figure()
# plt.imshow(model.W, interpolation='none')
# plt.colorbar()
# plt.title("HP connectivity matrix")
# plt.show()

In [7]:
######################################################################
######################################################################
######################################################################

# mat_input = np.where(np.random.rand(input_shape[0],input_shape[1])<0.5,0,1)
# mat_input = np.mean(plt.imread(np.random.choice(options.train)),axis=2) + np.random.uniform(-1,1,input_shape)

img_id = 7

# random uniform noise
#mat_input = np.mean(plt.imread(imgs[img_id]),axis=2) + np.random.uniform(-1,1,input_shape)

# occulsion
original_img = plt.imread(imgs[img_id])
print ("temp: ", original_img.shape)
original_img = np.mean(original_img, axis=2)
original_img = np.where(original_img < 0.5,0,1)

#
mat_input = original_img.copy()
mat_input[:10] = 0

#
mat_input = np.where(mat_input < 0.5,0,1)
#mat_input = np.where(mat_input < 0.5,-1,1)

### Update states ###
#Q: why do we only iterate once on the outer loops?!  
iteration = 1

output_async,e_list_async, states = model.predict(mat_input,
                                          original_img, 
                                          iteration,
                                          asyn=True)

#
output_sync,e_list_sync, states = model.predict(mat_input,
                                        original_img, 
                                        iteration,
                                        asyn=False)


temp:  (32, 32, 3)
(32, 32)
Starting asynchronous update with  1  iterations


100%|████████████████████████████████████████████████████████████████████| 1024/1024 [00:00<00:00, 3179.13it/s]


Iteration# 0 , Energy:  -719620.0
Iteration completed, update will now stop.
(32, 32)
Starting synchronous update with  1  iterations
Iteration# 0 , Energy:  -784776.0
Iteration completed, update will now stop.


In [6]:
plt.figure()
plt.subplot(131)
plt.imshow(states[0].reshape(32,32))
plt.subplot(132)
plt.imshow(states[1].reshape(32,32))
plt.subplot(133)
plt.imshow(states[2].reshape(32,32))
plt.show()

In [6]:
######################################################################
######################################################################
######################################################################
### Plotting Final Result###
fig = plt.figure(1)
fig.suptitle('Result with %i iteration' %iteration)

#
axs1 = fig.add_subplot(231)
axs1.set_title('Input')
axs1.imshow(mat_input*255,cmap='binary')

#
axs2 = fig.add_subplot(232)
axs2.set_title('Async Update')
axs2.imshow(output_async*255,cmap='binary')

#
axs3 = fig.add_subplot(233)
axs3.set_title('Sync Update')
axs3.imshow(output_sync*255,cmap='binary')

#
axs4 = fig.add_subplot(2,1,2)
axs4.plot(e_list_async)
axs4.plot(e_list_sync)
axs4.annotate(int(e_list_async[-1]),[len(e_list_async)-1,e_list_async[-1]])
axs4.annotate(int(e_list_sync[-1]),[len(e_list_sync)-1,e_list_sync[-1]])
axs4.set_ylabel('Energy')
axs4.set_xlabel('Iteration')
axs4.legend(['Async','Sync'])
axs4.grid(b=True,which='both')
axs4.set_xlim(0,max(len(e_list_async),len(e_list_sync))-1)

#
fig.canvas.draw_idle()
fig2 = plt.figure(2)
fig2.suptitle('Model Weights')
axs2 = fig2.gca()
axs2.imshow(model.W)
fig2.canvas.draw_idle()
#

plt.show()
#plt.pause(0.5)
#input("Press Enter to continue...")

/home/cat/anaconda3/envs/ai/lib/python3.7/site-packages/ipykernel_launcher.py:32: MatplotlibDeprecationWarning: The 'b' parameter of grid() has been renamed 'visible' since Matplotlib 3.5; support for the old name will be dropped two minor releases later.


In [ ]:
def energy(W, state):
    """
    Returns the energy for any input to the network.
    """
    return -0.5 * np.sum(np.multiply(np.outer(state, state), W))

#
img_id = 2

# occulsion
original_img = plt.imread(imgs[img_id])
print ("temp: ", original_img.shape)
original_img = np.mean(original_img, axis=2)
original_img = np.where(original_img < 0.5,-1,1)

# add noise
original_img = original_img + np.random.randint(low=-1, high=1, size=(original_img.shape[0], original_img.shape[1]))
original_img = np.clip(original_img, -1,1)

#
e = energy(model.W, original_img)
print (e)

temp:  (32, 32, 4)
-232016.0


In [ ]:
#########################################
#########################################
#########################################
def boolean_flip_random(data, 
                       noise_level):
    ''' Convert data into -1s and +1s     
    '''

    # select random numbers between 0 and 1 (uniformly dstirbuted)
    rnd = np.random.random(data.shape)
    
    # for all values < noise level flip them
    idx = np.where(rnd<noise_level)
        
    #
    data[idx] = data[idx]*-1
        
    #
    return data

#
def add_random_noise_flips(pattern,
                           noise_level):
    
    #This allows you to add noise to a pattern
    pattern = boolean_flip_random(pattern.copy(),
                                         noise_level)
    
    return pattern


# 
original_imgs = []
for img in imgs:
    original_img = plt.imread(img)
    original_img = np.mean(original_img, axis=2)
    original_img = np.where(original_img < 0.5,-1,1)
    #print ("original_img: ", np.unique(original_img))

    original_imgs.append(original_img.flatten())

original_imgs = np.array(original_imgs)
print ("original imgs: ", original_imgs.shape)

#
if False:
    plt.figure()
    ax=plt.subplot(121)
    plt.imshow(original_imgs[k])
    plt.colorbar()
    ax=plt.subplot(122)
    plt.imshow(noisy_img)
    plt.colorbar()
    
#noisy_imgs = np.array(noisy_imgs)
#print ("noisy imgs: ", noisy_imgs.shape)

#
# all_imgs = np.vstack((original_imgs, noisy_imgs))
# all_imgs = all_imgs.reshape(all_imgs.shape[0],-1)
# print ("all imgs: ", all_imgs.shape)

############################################################## 
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
#pca.fit(all_imgs)
pca.fit(original_imgs)
pca_res = pca.transform(original_imgs)

plt.figure()
for k in range(original_imgs.shape[0]):
    ax=plt.subplot(1,8,k+1)
    temp1 = original_imgs[k].reshape(32,32)
    temp2 = pca.inverse_transform(pca.transform(original_imgs[k].flatten()[None])).reshape(32,32)
    temp2 = np.where(temp2 < 0.5,-1,1)
    temp = np.vstack((temp1,temp2))
    plt.imshow(temp)
    
plt.show()
    

# 
plt.figure()
plt.scatter(pca_res[:,0],
            pca_res[:,1],
            edgecolor='black',
           alpha=.1)
#
for k in range(len(original_imgs)):
    plt.scatter(pca_res[k,0],
                pca_res[k,1], 
                label=str(k),
               color='red',
               s=100)

plt.legend()


pc1_minmax = [np.min(pca_res[:,0]), np.max(pca_res[:,0])]
pc2_minmax = [np.min(pca_res[:,1]), np.max(pca_res[:,1])]
print ("pc1: ", pc1_minmax, 
       "pc2: ", pc2_minmax)
plt.show()



original imgs:  (8, 1024)
pc1:  [-13.665967285937565, 21.21584734236046] pc2:  [-16.876654963291358, 20.78491182073399]


In [9]:
################################################################
######### FIT NOISE IMAGES BASED ON GRID OF PCA SPACE ##########
################################################################
#

def compute_surface_energy(idx,
                           pca, 
                           W,
                           pc1, 
                           pc2):
    
    #
    #pc_points = np.zeros((pc1.shape[0]*pc2.shape[0],2),'float32')
    surface_energy = np.zeros((pc1.shape[0],pc2.shape[0]),'float32')
             
    # loop over only assigned vals
    #for id1 in idx:
    for id1 in idx:
        
        # loop over all vals
        for id2 in range(pc2.shape[0]):
            
            pc1_ = pc1[id1]
            pc2_ = pc2[id2]
            res = pca.inverse_transform([pc1_,pc2_])
            res = np.where(res < 0.5,-1,1)
            #temp2 = np.where(temp2 < 0.5,-1,1)
            #surface_points.append(res)
            temp = energy(model.W, res)
            surface_energy[id1, id2] = temp

    return surface_energy
            
# make grid to sampel on\
grid_spacing = 1

# select all points
if True:
    pc1 = np.arange(pc1_minmax[0]*1.2, pc1_minmax[1]*1.2, grid_spacing)
    pc2 = np.arange(pc2_minmax[0]*1.2, pc2_minmax[1]*1.2, grid_spacing)

# select a singl epoint
else:
    pt_id = 0
    buffer = 1
    #print (res)
    pc1 = np.arange(pca_res[pt_id,0]-buffer, pca_res[pt_id,0]+buffer, grid_spacing)
    pc2 = np.arange(pca_res[pt_id,1]-buffer, pca_res[pt_id,1]+buffer, grid_spacing)
    #pc2 = np.arange(pc2_minmax[0]*1.2, pc2_minmax[0]*0.8, grid_spacing)


#
if True:
    n_cores=30
    idx_array = np.array_split(np.arange(pc1.shape[0]), n_cores)
    #print ("idx array: ", idx_array)
    res = parmap.map(compute_surface_energy,
                     idx_array,
                     pca,
                     model.W,
                     pc1,
                     pc2,                     
                     pm_processes=n_cores,
                     pm_pbar=True)
    
    #
    surface_energy = np.array(res).sum(0)
    
    
else:
    idx = np.arange(len(pc1))
    surface_energy = compute_surface_energy(idx,
                                            pca,
                                            model.W,
                                            pc1,
                                            pc2)


#
print ("surface_energy: ", surface_energy.shape)
    
#
pc_points = []
for pc1_ in pc1:
    for pc2_ in pc2:
        pc_points.append([pc1_,pc2_])
#
pc_points = np.vstack(pc_points)
print (pc_points.shape)

print ("DONE...")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:02<00:00, 13.46it/s]

surface_energy:  (42, 46)
(1932, 2)
DONE...


In [13]:
################################################################
################################################################
################################################################
from matplotlib import cm

original_pts = pca.transform(original_imgs.reshape(original_imgs.shape[0],-1))

# get energy of original poitns
# NOTE: Must use reconstructed vecotr just like for the grid sampling in PCA space
#
original_energy = []
for k in range(len(original_imgs)):
    temp = original_imgs[k].flatten()
    temp_pca = pca.inverse_transform(pca.transform(temp[None]))
    temp_pca = np.where(temp_pca < 0.5,-1,1)
    temp = energy(model.W, temp_pca)
    #temp = energy(model.W, temp_pca)
    original_energy.append(temp)
    
#
fig = plt.figure()
ax = plt.axes(projection="3d")

Xs = pc_points[:,0]
Ys = pc_points[:,1]
Zs = surface_energy.flatten()
print (Xs.shape, Ys.shape, Zs.shape)

surf = ax.plot_trisurf(Xs, 
                       Ys, 
                       Zs,
                       cmap=cm.viridis, 
                       linewidth=0,
                      alpha=.6)
fig.colorbar(surf)

#ax.xaxis.set_major_locator(MaxNLocator(5))
#ax.yaxis.set_major_locator(MaxNLocator(6))
#ax.zaxis.set_major_locator(MaxNLocator(5))

# ax.scatter3D(pc_points[:,0],
#              pc_points[:,1],
#              surface_energy,
#              c=surface_energy)

ax.scatter3D(original_pts[:,0],
             original_pts[:,1],
             original_energy,
             c='red',
             s=500)

for k in range(len(LABELS)):
    ax.text(original_pts[k,0],
               original_pts[k,1],
               original_energy[k],
               LABELS[k],
           fontsize=20)

plt.show()


(1932,) (1932,) (1932,)


In [157]:
##############################################################
##################### 2D PLOTS ###############################
##############################################################

# recompute original pts energy but in PCA denoised space
original_pts = pca.transform(original_imgs.reshape(original_imgs.shape[0],-1))

# get energy of original poitns
# NOTE: Must use reconstructed vecotr just like for the grid sampling in PCA space
#
original_energy = []
for k in range(len(original_imgs)):
    temp = original_imgs[k].flatten()
    temp_pca = pca.inverse_transform(pca.transform(temp[None]))
    temp_pca = np.where(temp_pca < 0.5,-1,1)
    temp = energy(model.W, temp_pca)
    #temp = energy(model.W, temp_pca)
    original_energy.append(temp)
    
#
fig = plt.figure()
ax = plt.axes()

ax.scatter(pc_points[:,0],
           pc_points[:,1],
             #surface_energy,
             c=surface_energy)
#

for k in range(len(LABELS)):
    ax.scatter(original_pts[k,0],
               original_pts[k,1],
               #original_energy,
               c='red',
               #label=LABELS[k],
               s=500)
    ax.text(original_pts[k,0],
               original_pts[k,1],
               LABELS[k])
    
#plt.legend()
plt.show()

(35,)
(38,)
1330


## NETWORK ANALYSIS BELOW

In [174]:
#######################################################
#######################################################
#######################################################
def compute_networkx_custom2_excitatory(transition_matrix,
                           thresh):

    T = transition_matrix.copy()
    np.fill_diagonal(T, 0)

    # 
    #thresh = 1
    idx0 = np.where(T<thresh)
    idx1 = np.where(T>=thresh)
    T[idx0]=0
    T[idx1]=1
    print ("# excitatory connections: ", idx1[0].shape)

    #
    G = nx.from_numpy_matrix(T)
    # G = nx.DiGraph(T)
    idx = list(nx.isolates(G))
    #G.remove_nodes_from(idx)
    # G.remove_nodes_from(idx)

    #pos = nx.circular_layout(G)
    #pos = nx.spring_layout(G)

    return G, T

def compute_networkx_custom2_inhibitory(transition_matrix,
                                      thresh):

    T = transition_matrix.copy()
    np.fill_diagonal(T, 0)

    # 
    print ("threshold: ", thresh)
    idx0 = np.where(T>thresh)
    idx1 = np.where(T<=thresh)
    print ("# inhibitory connections: ", idx1[0].shape)
    T[idx0]=0
    T[idx1]=1

    #
    G = nx.from_numpy_matrix(T)
    # G = nx.DiGraph(T)
    
    # option to remove isolates
    if False:
        idx = list(nx.isolates(G))

        G.remove_nodes_from(idx)
        # G.remove_nodes_from(idx)

    #pos = nx.circular_layout(G)
    #pos = nx.spring_layout(G)

    return G, T

############################################
import networkx as nx
        
print (" model: ", model.W.shape)
#print (np.unique(model.W))

model_norm = model.W/np.max(model.W)
print (" model unique connectivity values: ", np.unique(model_norm, return_counts=True))

#inhibitory = True
excitatory = False
if excitatory:
    thresh = 0.5
    G_exc,T_exc = compute_networkx_custom2_excitatory(model_norm.copy(), 
                                        thresh)
#
else:
    thresh = -0.5
    G_inh,T_inh = compute_networkx_custom2_inhibitory(model_norm.copy(), 
                                        thresh)

########################################
######## PLOT NEURAL-SPACE LOC #########
########################################

# #
#n_unique = np.unique(T).shape[0]
#sizes2 = np.zeros(n_unique)+25

# 
#sizes2[30]=500

plt.figure()
plt.imshow(T)
plt.colorbar()
plt.show()

 model:  (1024, 1024)
 model unique connectivity values:  (array([-1.  , -0.75, -0.5 , -0.25,  0.  ,  0.25,  0.5 ,  0.75,  1.  ]), array([   276,   5302,  31334,  96518, 193346, 262716, 248546, 159552,
        50986]))
threshold:  -0.5
# inhibitory connections:  (36912,)


In [142]:
plt.figure()
idx = np.where(model_norm>0)
model_norm2 = model_norm.copy()
model_norm2[idx]=np.nan
plt.imshow(model_norm2,
          )
plt.colorbar()
plt.show()

In [175]:
plt.figure()
    
G = G_exc.copy()
    
random_pos = nx.random_layout(G, seed=0)
pos = nx.spring_layout(G, pos=random_pos)
print (G.number_of_nodes(), len(pos))

#
weights = np.zeros(len(random_pos))+0.01
nx.draw(G, 
        pos, 
        #node_color=np.arange(sizes2.shape[1]), 
        node_size=15, 
        width=weights,
        cmap=plt.cm.viridis)

plt.show()

1024 1024
DONE...


In [122]:
unique, counts = np.unique(model_norm, return_counts=True)
plt.figure()
plt.bar(unique, counts,0.24)
plt.xlabel("connectivity strength (-1,+1)")
plt.ylabel("# of connections")
plt.show()


In [180]:
###################################################
###################################################
###################################################
def get_degree_distribution(G):
    degrees = []
    for d in G.degree:
        degrees.append(d)

    # 
    degrees  = np.vstack(degrees)
    print ("Degrees: ", degrees)
    
    idx = np.argsort(degrees[:,0])
    degrees = degrees[idx]
    print ("degrees ordered: ", degrees)

    ds = degrees[:,1]
    y = np.histogram(ds, bins=np.arange(0,1024,2))

    
    return y[1][:-1],y[0]

#
plt.figure()
labels = ["G_exc", "G_inh"]
ctr=0
for G in [G_exc, G_inh]:
    x, y = get_degree_distribution(G)

    #
    print (y)
    idx = np.argsort(y)[::-1]
    print (y[idx])
    y = y[idx]
    #x=x[idx]

    plt.scatter(x,y+0.1,
            #color=cmap(ctr)
                label=labels[ctr]
            )
    #plt.semilogy()
    ctr+=1
plt.legend()
plt.xlabel("node")
plt.ylabel("degree")
plt.semilogx()
plt.title("Degree of connections (>0.5 and <-0.5)")
plt.show()
#print (pr[110])

Degrees:  [[   0  720]
 [   1  720]
 [   2  720]
 ...
 [1021  463]
 [1022  463]
 [1023  463]]
degrees ordered:  [[   0  720]
 [   1  720]
 [   2  720]
 ...
 [1021  463]
 [1022  463]
 [1023  463]]
[  0   0   0   0   0   0   0   2   0   0   0   0   2   0   0   0   0   0
   0   1   0   4   2   0   0   0   0   0   1   0   0   0   1   0   4  10
   4   3   4   0   5   3   0   0   3   7   0   0   0   9   9   3  10   0
   6   0   1   0   0  13   0   0   2   0   0   0   0   0   0   0   3   0
   2   0   0   3   0   1  12   0   5   0   9   0   1   0   9   0   0   1
   0   0   2   0   2   0   0   2  11   1   0   0   0   4   3   5   0   0
   1  40   0   0   0   1   0   7   0   4   0   0   0   0   0   0   0   0
   0  30   0   0   0   0   0   6   0   0   0   0   0   0   0   0   4   0
   0   3   0   0   0   5   0   0   0   5   4   0   0   2  22   0   0   0
   0   0   0   0   0   0  13   0   0   2   3   0   0   0  12   0   0   0
   0  34   0   0  11   0   0   0   2   3   0   1   9   4   0   0   8   9
 